# ```torch.nn```到底是什么

PyTorch 提供设计精美的模块和类<u>```torch.nn```</u>，<u>```torch.optim```</u>，<u>```Dataset```</u>和<u>```DataLoader```</u>神经网络。 为了充分利用它们的功能并针对您的问题对其进行自定义，您需要真正了解它们在做什么。 为了建立这种理解，我们将首先在 MNIST 数据集上训练基本神经网络，而无需使用这些模型的任何功能。 我们最初将仅使用最基本的 PyTorch 张量函数。 然后，我们将一次从```torch.nn```，```torch.optim```，```Dataset```或```DataLoader```中逐个添加一个函数，以准确显示每个函数，以及如何使代码更简洁或更有效。 灵活。

## MNIST 数据集

我们将使用经典的``` MNIST ```数据集，该数据集由手绘数字的黑白图像组成（0 到 9 之间）。

我们将使用```pathlib```处理路径（Python 3 标准库的一部分），并使用```requests```下载数据集。 我们只会在使用模块时才导入它们，因此您可以确切地看到每个位置上正在使用的模块。

In [4]:
from pathlib import Path
import requests

DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "https://github.com/pytorch/tutorials/raw/master/_static/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)

该数据集为 numpy 数组格式，并已使用 ```pickle```（一种用于序列化数据的 python 特定格式）存储。

In [5]:
import pickle
import gzip

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

EOFError: Ran out of input

每个图像为 ```28 x 28```，并存储为长度为 ```784 = 28x28``` 的扁平行。 让我们来看一个； 我们需要先将其重塑为 2d。

In [ ]:
from matplotlib import pyplot
import numpy as np

pyplot.imshow(x_train[0].reshape((28, 28)), cmap="gray")
print(x_train.shape)

NameError: name 'x_train' is not defined

PyTorch 使用 ```torch.tensor``` 而不是 numpy 数组，因此我们需要转换数据。

In [6]:
import torch

x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)
n, c = x_train.shape
x_train, x_train.shape, y_train.min(), y_train.max()
print(x_train, y_train)
print(x_train.shape)
print(y_train.min(), y_train.max())

NameError: name 'x_train' is not defined

## 从零开始的神经网络（没有 ```torch.nn``` ）

首先，我们仅使用 PyTorch 张量操作创建模型。 我们假设您已经熟悉神经网络的基础知识。 （如果不是，则可以在 <u>```course.fast.ai```</u> 中学习它们）。

PyTorch 提供了创建随机或零填充张量的方法，我们将使用它们来为简单的线性模型创建权重和偏差。 这些只是常规张量，还有一个非常特殊的附加值：我们告诉 PyTorch 它们需要梯度。 这使 PyTorch 记录了在张量上完成的所有操作，因此它可以在反向传播时自动计算的梯度！

**对于权重，我们在初始化之后设置 <font color="orange">requires_grad </font>，因为我们不希望该步骤包含在梯度中。 （请注意，PyTorch 中的尾随_表示该操作是原地执行的。）**

注意

我们在这里用 <font color="orange">Xavier</font> 初始化（通过乘以 $1 / sqrt(n)$）来初始化权重。

In [8]:
import math

weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

由于 PyTorch 具有自动计算梯度的功能，我们可以将任何标准的 Python 函数（或可调用对象）用作模型！ 因此，我们只需编写一个普通矩阵乘法和广播加法即可创建一个简单的线性模型。 我们还需要激活函数，因此我们将编写并使用 ```log_softmax``` 。 请记住：尽管 PyTorch 提供了许多预写的损失函数，激活函数等，但是您可以使用纯 Python 轻松编写自己的函数。 PyTorch 甚至会自动为您的函数创建快速 GPU 或向量化的 CPU 代码。

In [9]:
def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1)

def model(xb):
    return log_softmax(xb @ weights + bias)

在上面， ```@``` 代表点积运算。 我们将对一批数据（在本例中为 64 张图像）调用函数。 这是一个正向传播。 请注意，由于我们从随机权重开始，因此在这一阶段，我们的预测不会比随机预测更好。

In [10]:
bs = 64  # batch size

xb = x_train[0:bs]  # a mini-batch from x
preds = model(xb)  # predictions
preds[0], preds.shape
print(preds[0], preds.shape)


NameError: name 'x_train' is not defined